## MSER 특징 검출

In [26]:
# 0903.py
import numpy as np
import cv2
 
src = cv2.imread('./data/chessBoard.jpg')
gray= cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

#1
mserF = cv2.MSER_create(10)  # cv2.MSER.create(10)
kp= mserF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color=(0,0,255))
cv2.imshow('dst',  dst)

#2
dst2 = dst.copy()
regions, bboxes = mserF.detectRegions(gray)
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
cv2.polylines(dst2, hulls, True, (0, 255, 0))
cv2.imshow('dst2',  dst2)

#3
dst3 = dst.copy()
for i, pts in enumerate(regions):
    box = cv2.fitEllipse(pts)
    cv2.ellipse(dst3, box,  (255,0,0),1)
    x, y, w, h = bboxes[i]
    cv2.rectangle(dst3, (x, y), (x+w, y+h), (0,255,0))     
cv2.imshow('dst3',  dst3)
cv2.waitKey()
cv2.destroyAllWindows()


len(kp)= 202


In [8]:
# 0906.py
import cv2
import numpy as np
#1
def distance(f1, f2):    
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1)**2+ (y2 - y1)**2)

def filteringByDistance(kp, distE=0.5):
    size = len(kp)
    mask = np.arange(1,size+1).astype(np.bool8) # all True   
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else: # True
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2)<distE:
                    mask[j] = False
    np_kp = np.array(kp)
    return list(np_kp[mask])

#2
src = cv2.imread('./data/cornerTest.jpg')
##src = cv2.imread('./data/chessBoard.jpg')
gray= cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0.0)

##orbF = cv2.ORB_create()           # HARRIS_SCORE
orbF = cv2.ORB_create(scoreType=1)  # FAST_SCORE
kp= orbF.detect(gray)
print('len(kp)=', len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color=(0,0,255))   
cv2.imshow('dst',  dst)

#3
kp = sorted(kp, key=lambda f: f.response, reverse=True)
filtered_kp = list(filter(lambda f: f.response>50, kp))
filtered_kp = filteringByDistance(kp, 10)
print('len(filtered_kp)=', len(filtered_kp))

kp, des = orbF.compute(gray, filtered_kp)
print('des.shape=', des.shape)
print('des=', des)

#4
dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color=(0,0,255))  
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x, y), (size, size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0,255,0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size/2), (255,0,0), 2)
cv2.imshow('dst2',  dst2)
cv2.waitKey()
cv2.destroyAllWindows()


len(kp)= 63
len(filtered_kp)= 8
des.shape= (8, 32)
des= [[ 72  48  56  96  33  77  81  16 105 168  52   8 159  23  64  50 132 213
  221   8 136  96 240   0 195 249  33  48  66 128  67  35]
 [104  48  56  96  33  77  81  16 105 168  52   8 159  23  64  50 132 212
   93   8 136  96 240   0 195 217  33  48  66 128  66  35]
 [  2 141   2 158  20 131 174 230  22   0  90   2  96 234  48  64  89 106
   19 193   2 184   8 255  40  38 138 128  20  10  48 136]
 [  2 141   2 158  20 131 174 230  22   0  88   2  32 234  48  64  89 106
   19 193   2 184  10 255  40  38 138 136  20  10  48 136]
 [  2 141   2 158  20 131 174 230  22   1  88   2  32 234  48  64  89 106
   19 193   2 184   8 255  40  38 138 128  20  10  48 136]
 [  2 141   2 158  20 131 174 230  22   0  90   2  96 234  48  64  89 106
   19 193   2 184  10 255  40  38 138 128  20  10 176 136]
 [  2 140   2 154  84 131 170 238  22   1  88   2  96 232  56  64 121 106
   19 129  34 186  10 255  40  38 155 136  21  10  48 136]
 [  2 141   2

In [28]:
# 0918.py
import cv2
from   matplotlib import pyplot as plt

#1
src = cv2.imread('./data/people.png')
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

#2
loc1, weights1 = hog.detect(src)
print('len(loc1)=',len(loc1))
dst1 = src.copy()
w, h = hog.winSize
for pt in loc1:
    x, y = pt
    cv2.rectangle(dst, (x, y), (x+w, y+h), (255,0,0),2)
cv2.imshow('dst1',  dst1)

#2
dst2 = src.copy()
loc2, weights2 = hog.detectMultiScale(src)
print('len(loc2)=',len(loc2))
for rect in loc2:
    x, y, w, h = rect
    cv2.rectangle(dst2, (x, y), (x+w, y+h), (0,255,0),2)
cv2.imshow('dst2',  dst2)

#3
dst3 = src.copy()
loc3, weights3 = hog.detectMultiScale(src, winStride=(1,1), padding=(8,8))
print('len(loc3)=',len(loc3))
print('weights3=', weights3)
for i, rect in enumerate(loc3):
    x, y, w, h = rect
    if weights3[i]>0.5:
        cv2.rectangle(dst3, (x, y), (x+w, y+h), (0,0,255),2)
    else:
        cv2.rectangle(dst3, (x, y), (x+w, y+h), (255,0,0),2)
    
cv2.imshow('dst3',  dst3)
cv2.waitKey()
cv2.destroyAllWindows()


len(loc1)= 0
len(loc2)= 3
len(loc3)= 6
weights3= [[3.82788096]
 [2.49923779]
 [2.4061097 ]
 [1.13816947]
 [0.64257275]
 [0.37325622]]


In [18]:
sorted

<function sorted(iterable, /, *, key=None, reverse=False)>